In [1019]:
import pandas as pd
import numpy as np

In [1020]:
weeks = pd.read_csv('./data2021/games.csv')['week'].max()

In [1021]:
player_stats = pd.read_csv("merged_df.csv").drop('Unnamed: 0', axis='columns')

In [1022]:
play_data = pd.read_csv("./data2021/plays.csv")

In [1023]:
player_stats.head()

,Acceleration,Age,Agility,Awareness,BC Vision,Block Shedding,Break Sack,Break Tackle,Carrying,Catch In Traffic,...,Zone Coverage,birthDate,collegeName,displayName,height,nflId,position,updatedName,updatedName2,weight
0,93,27,97,90,74,59,13,63,54,50,...,93,1990-09-10,Washington,Desmond Trufant,72,2539334,CB,Desmond Trufant,Desmond Trufant,190
1,93,29,94,82,79,57,15,65,64,45,...,83,1988-11-01,Southeastern Louisiana,Robert Alford,70,2539653,CB,Robert Alford,Robert Alford,186
2,89,26,92,87,82,60,15,75,65,60,...,81,1991-12-18,Purdue,Ricardo Allen,69,2543850,SS,Ricardo Allen,Ricardo Allen,186
3,90,23,87,92,32,72,12,14,56,45,...,88,1994-11-04,Louisiana State,Deion Jones,73,2555162,MLB,Deion Jones,Deion Jones,227
4,87,25,87,82,41,68,12,16,45,45,...,80,1993-07-01,Minnesota,De'Vondre Campbell,75,2555255,OLB,De'Vondre Campbell,De'Vondre Campbell,232


In [1024]:
player_stats.Team.unique()

array(['Falcons', 'Eagles', 'Bills', 'Ravens', 'Steelers', 'Browns',
       'Raiders', 'Bengals', 'Colts', 'Titans', 'Dolphins', 'Texans',
       'Patriots', 'Buccaneers', 'Saints', 'Jaguars', 'Giants',
       'Redskins', 'Cardinals', 'Cowboys', 'Panthers', 'Bears', 'Packers',
       'Jets', 'Lions', '49ers', 'Rams', 'Chargers', 'Vikings', 'Chiefs',
       'Seahawks', 'Broncos'], dtype=object)

In [1025]:
# Get name to match play_data format
player_stats["abrev_name"] = player_stats.displayName.apply(lambda x: str(x)[0]+ "."+"".join(str(x).split(" ")[1:]))
player_stats.abrev_name = player_stats.abrev_name.apply(lambda x: str(x).lower())

In [1026]:
player_stats.abrev_name

0        d.trufant
1         r.alford
2          r.allen
3          d.jones
4       d.campbell
          ...     
963       j.reaves
964          t.way
965        m.smith
966    m.mccaffrey
967     d.williams
Name: abrev_name, Length: 968, dtype: object

In [1027]:
team_mapping = {"PHI": "Eagles", "BUF": "Bills", "ATL": "Falcons", "PIT": "Steelers", "CLE": "Browns", "CIN": "Bengals", "IND": "Colts", "TEN": "Titans", "MIA": "Dolphins", "BAL": "Ravens", "NE": "Patriots", "HOU": "Texans", "JAX": "Jaguars", "NYG": "Giants", "NO": "Saints", "TB": "Buccaneers", "WAS": "Commanders", "ARI": "Cardinals", "CAR": "Panthers", "DAL": "Cowboys", "GB": "Packers", "CHI": "Bears", "NYJ": "Jets", "DET": "Lions", "OAK": "Raiders", "LA": "Rams", "MIN": "Vikings", "LAC": "Chargers", "KC": "Chiefs", "SF": "49ers", "DEN": "Broncos", "SEA": "Seahawks", "LV": "Raiders"}

In [1028]:
play_data['Team'] = play_data.possessionTeam.apply(lambda x: team_mapping[str(x)])

In [1029]:
play_data.possessionTeam.unique()

array(['ATL', 'PHI', 'PIT', 'CLE', 'CIN', 'IND', 'TEN', 'MIA', 'BAL',
       'BUF', 'NE', 'HOU', 'JAX', 'NYG', 'NO', 'TB', 'WAS', 'ARI', 'CAR',
       'DAL', 'GB', 'CHI', 'NYJ', 'DET', 'OAK', 'LA', 'MIN', 'LAC', 'KC',
       'SF', 'DEN', 'SEA'], dtype=object)

In [1030]:
# Remove time from play description
playDescText = play_data.playDescription.apply(lambda x: " ".join(str(x).split()[1:]))
# Remove formation from play description
playDescText = playDescText.apply(lambda x: "".join(x.split(")")[1:]) if str(x).startswith("(") else x)
playDescText

0        M.Ryan pass short right to J.Jones pushed ob a...
1        M.Ryan pass incomplete short right to C.Ridley...
2          M.Ryan pass incomplete short left to D.Freeman.
3         M.Ryan pass deep left to J.Jones to PHI 6 for...
4         M.Ryan pass incomplete short right to D.Freeman.
                               ...                        
19234    J.Johnson pass incomplete short left [K.Correa...
19235     B.Gabbert pass incomplete short left to T.Tay...
19236     P.Rivers pass deep right to K.Allen to BLT 24...
19237    P.Rivers pass incomplete deep middle to K.Alle...
19238     L.Jackson pass incomplete short left to M.Cra...
Name: playDescription, Length: 19239, dtype: object

In [1031]:
# People who received the ball
receivers = playDescText.map(lambda x: str(x).lower().split(" ")[str(x).lower().split(" ").index("to")+1] if ("pass" in str(x).lower().split(" ") and "to" in str(x).lower().split(" ")) & ("intercept" not in str(x).lower()) else np.nan)
# Remove full stop from some names at end
receivers.loc[receivers.map(lambda x: str(x)[-1]) == "."] = receivers.loc[receivers.map(lambda x: str(x)[-1]) == "."].apply(lambda x: "".join(str(x)[:-1]))
# 2 values have NO # instead of names, so we can remove these
receivers.loc[receivers=="no"] = np.nan

In [1032]:
play_data['abrev_name'] = receivers # column representing who received
receiver_stats = play_data.merge(player_stats, how='left', left_on=['Team','abrev_name'], right_on=['Team','abrev_name'])
receiver_stats = receiver_stats.rename(columns={'abrev_name': "receiver"})

In [1033]:
receiver_stats.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,...,Zone Coverage,birthDate,collegeName,displayName,height,nflId,position,updatedName,updatedName2,weight
0,2018090600,75,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018090600,146,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,...,25.0,1994-12-20,Alabama,Calvin Ridley,73,2560854.0,WR,Calvin Ridley,Calvin Ridley,190.0
2,2018090600,168,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,...,15.0,1992-03-15,Florida State,Devonta Freeman,68,2543583.0,RB,Devonta Freeman,Devonta Freeman,206.0
3,2018090600,190,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018090600,256,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,...,15.0,1992-03-15,Florida State,Devonta Freeman,68,2543583.0,RB,Devonta Freeman,Devonta Freeman,206.0


In [1034]:
#Drop non-Madden players and anomalies
receiver_stats.dropna(axis='rows', subset='nflId', inplace=True)
receiver_stats.reset_index(drop=True, inplace=True)

In [1035]:
len(receiver_stats['nflId'].unique())

339

In [1036]:
feature_columns = ['Age', 'Height', 'Weight', 'Overall', 'Speed',
 'Acceleration', 'Agility', 'Change of Dir', 'Strength', 'Jumping',
 'Awareness', 'Carrying', 'Break Tackle', 'Juke Move', 'Spin Move',
 'Trucking', 'Stiff Arm', 'BC Vision', 'Catching', 'Catch In Traffic',
 'Spec Catch', 'Release', 'Short RR', 'Medium RR', 'Deep RR',
 'Throw Power', 'Throw Acc Short', 'Throw Acc Mid', 'Throw Acc Deep',
 'Throw Under Pressure', 'Throw On The Run', 'Play Action', 'Break Sack',
 'Run Block', 'Run Block Power', 'Run Block Finesse', 'Pass Block',
 'Pass Block Power', 'Pass Block Finesse', 'Impact Blocking',
 'Lead Blocking', 'Tackle', 'Hit Power', 'Pursuit', 'Man Coverage',
 'Zone Coverage', 'Press', 'Play Recognition', 'Power Moves',
 'Finesse Moves', 'Block Shedding', 'Kick Power', 'Kick Accuracy',
 'Kick Return', 'Stamina', 'Injury', 'Toughness', 'Years Pro']

In [1037]:
total_yards_by_stats = pd.DataFrame({'nflId': receiver_stats['nflId'].astype('int32').unique()})

In [1038]:
total_yards_by_stats['total_yards'] = total_yards_by_stats['nflId'].map(lambda x: receiver_stats[receiver_stats['nflId'] == x]['offensePlayResult'].sum() / weeks)

In [1039]:
total_yards_by_stats = total_yards_by_stats.merge(player_stats[feature_columns + ['nflId']], how='inner', on='nflId')
total_yards_by_stats

,nflId,total_yards,Age,Height,Weight,Overall,Speed,Acceleration,Agility,Change of Dir,...,Power Moves,Finesse Moves,Block Shedding,Kick Power,Kick Accuracy,Kick Return,Stamina,Injury,Toughness,Years Pro
0,2560854,48.529412,23,73,190,77,93,93,95,85,...,10,10,24,19,16,68,90,86,76,0
1,2543583,1.352941,26,68,206,91,92,91,95,91,...,10,10,26,10,10,76,96,94,81,4
2,2552600,44.647059,25,72,198,84,91,90,93,85,...,10,10,25,41,31,80,91,90,78,3
3,2506467,9.411765,35,66,190,83,86,90,94,86,...,10,10,20,14,11,98,84,87,90,13
4,2533040,50.000000,29,74,210,82,87,88,87,74,...,10,10,60,64,49,77,93,92,85,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,2558833,0.294118,24,74,253,65,82,89,76,59,...,73,61,69,25,25,40,92,95,88,1
335,2556273,0.000000,24,72,221,59,92,91,81,65,...,10,10,29,28,22,72,86,89,87,2
336,310,0.294118,33,76,217,89,72,81,69,56,...,10,10,25,25,25,10,92,98,94,10
337,2558875,0.294118,23,76,232,65,81,87,85,58,...,22,16,26,22,20,10,85,86,84,1


In [1040]:
total_yards_by_stats.describe()

,nflId,total_yards,Age,Height,Weight,Overall,Speed,Acceleration,Agility,Change of Dir,...,Power Moves,Finesse Moves,Block Shedding,Kick Power,Kick Accuracy,Kick Return,Stamina,Injury,Toughness,Years Pro
count,3.390000e+02,339.000000,339.000000,339.000000,339.000000,339.000000,339.000000,339.000000,339.000000,339.000000,...,339.000000,339.000000,339.000000,339.000000,339.000000,339.000000,339.000000,339.000000,339.000000,339.000000
mean,2.430801e+06,16.271907,25.837758,73.221239,218.852507,76.469027,86.560472,88.982301,85.802360,70.693215,...,13.761062,14.106195,34.106195,22.241888,20.631268,51.179941,88.737463,87.572271,81.280236,3.548673
std,5.075036e+05,19.271443,3.324678,2.931368,25.396188,7.880097,5.808968,4.029438,6.996567,13.654452,...,9.234558,9.390316,10.906295,8.788468,8.475322,27.683013,4.004300,4.139212,7.668497,3.225246
min,3.100000e+02,-0.647059,21.000000,66.000000,160.000000,59.000000,61.000000,64.000000,63.000000,15.000000,...,5.000000,5.000000,9.000000,10.000000,10.000000,6.000000,78.000000,71.000000,50.000000,0.000000
25%,2.540082e+06,2.176471,23.000000,71.000000,200.000000,70.500000,83.000000,88.000000,82.000000,64.000000,...,10.000000,10.000000,26.000000,18.000000,15.000000,25.000000,86.000000,85.000000,77.000000,1.000000
50%,2.552652e+06,9.352941,25.000000,73.000000,215.000000,76.000000,88.000000,90.000000,87.000000,74.000000,...,10.000000,10.000000,33.000000,21.000000,20.000000,55.000000,89.000000,88.000000,82.000000,3.000000
75%,2.558136e+06,24.029412,28.000000,76.000000,238.500000,82.000000,90.000000,91.000000,91.000000,80.000000,...,12.500000,14.500000,41.000000,25.000000,22.000000,76.000000,91.000000,90.000000,86.000000,5.000000
max,2.561482e+06,94.823529,41.000000,80.000000,277.000000,99.000000,98.000000,97.000000,98.000000,95.000000,...,74.000000,77.000000,75.000000,96.000000,89.000000,98.000000,99.000000,98.000000,99.000000,18.000000


In [1041]:
X = total_yards_by_stats[feature_columns]
X.shape

(339, 58)

In [1042]:
Y = total_yards_by_stats['total_yards']
Y.shape

(339,)

In [1043]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, random_state=0)

In [1044]:
select_k_best=True

In [1045]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression

if select_k_best:
    k_best = SelectKBest(mutual_info_regression, k=10).fit(X_train, Y_train)
    X_train = k_best.transform(X_train)
    X_test = k_best.transform(X_test)
    print(k_best.get_feature_names_out())

['Overall' 'BC Vision' 'Catching' 'Catch In Traffic' 'Spec Catch'
 'Release' 'Short RR' 'Medium RR' 'Deep RR' 'Run Block Power']


In [1046]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, Y_train)
reg.score(X_test, Y_test)

0.5281324957522062

In [1047]:
pd.DataFrame(reg.predict(X_test)).describe()

,0
count,68.000000
mean,17.213952
std,12.935356
min,-3.814241
25%,6.274098
50%,16.483769
75%,27.082421
max,52.154451


In [1048]:
from sklearn import svm
reg = svm.SVR().fit(X_train, Y_train)
reg.score(X_test, Y_test)

0.3905056628078656

In [1049]:
pd.DataFrame(reg.predict(X_test)).describe()

,0
count,68.000000
mean,11.815207
std,8.005098
min,2.342869
25%,4.355456
50%,10.087047
75%,18.076124
max,27.880589


In [1050]:
from sklearn import tree
reg = tree.DecisionTreeRegressor().fit(X_train, Y_train)
reg.score(X_test, Y_test)

0.11118355263048874

In [1051]:
pd.DataFrame(reg.predict(X_test)).describe()

,0
count,68.000000
mean,16.528547
std,19.272171
min,0.000000
25%,3.058824
50%,8.941176
75%,22.897059
max,94.823529


In [1052]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor().fit(X_train, Y_train)
reg.score(X_test, Y_test)

0.4226552250965263

In [1053]:
pd.DataFrame(reg.predict(X_test)).describe()

,0
count,68.000000
mean,16.929645
std,15.185293
min,2.293529
25%,6.201029
50%,11.051765
75%,26.015000
max,81.439412


In [1054]:
from sklearn.neural_network import MLPRegressor
reg = MLPRegressor(learning_rate_init = 0.01, learning_rate='adaptive').fit(X_train, Y_train)
reg.score(X_test, Y_test)

0.2992383702236592

In [1055]:
pd.DataFrame(reg.predict(X_test)).describe()

,0
count,68.000000
mean,14.431329
std,10.289362
min,-1.947076
25%,6.303170
50%,12.150352
75%,23.550232
max,34.384364
